In [2]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

if not firebase_admin._apps:
    
    cred = credentials.Certificate('nope.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sistema-rec-dmi-default-rtdb.firebaseio.com/'
    })
ref = db.reference('/base_usuarios')
ref2 = db.reference('/base_materias')

datos_firebase = ref.get()
datos_firebase2 = ref2.get()

df_firebase = pd.DataFrame(datos_firebase)
df_materias = pd.DataFrame(datos_firebase2)

rating_columns = [
    'electiva',
    'general_rating',
    'method_rating',
    'personalGrowth_rating',
    'rec_rating',
    'teacher_rating',
    'time_rating'
]

prefixes = ['first', 'second', 'third']

df_ratings_list = []

for indice, fila in df_firebase.iterrows():
    determinante = fila['determinante_electivas']
    
    for i, prefix in enumerate(prefixes, start=1):
        electiva_col = f'{prefix}_electiva'
        general_col = f'{prefix}_general_rating'
        method_col = f'{prefix}_method_rating'
        personalGrowth_col = f'{prefix}_personalGrowth_rating'
        rec_col = f'{prefix}_rec_rating'
        teacher_col = f'{prefix}_teacher_rating'
        time_col = f'{prefix}_time_rating'
        
        electiva = fila[electiva_col]
        
        if electiva.lower() != 'no':
            nueva_fila = {
                'electiva': electiva,
                'general_rating': fila[general_col] if fila[general_col] != '' else None,
                'method_rating': fila[method_col] if fila[method_col] != '' else None,
                'personalGrowth_rating': fila[personalGrowth_col] if fila[personalGrowth_col] != '' else None,
                'rec_rating': fila[rec_col] if fila[rec_col] != '' else None,
                'teacher_rating': fila[teacher_col] if fila[teacher_col] != '' else None,
                'time_rating': fila[time_col] if fila[time_col] != '' else None,
            }
            
            df_temporal = pd.DataFrame([nueva_fila], columns=rating_columns)
            df_ratings_list.append(df_temporal)

df_ratings = pd.concat(df_ratings_list, ignore_index=True)
df_ratings = df_ratings.dropna()

def find_similar_electives(electiva_seleccionada, top_n=3):
    selected_elective_data = df_ratings[df_ratings['electiva'] == electiva_seleccionada]
    selected_elective_data = selected_elective_data.drop(columns=['electiva'])
    similarity_scores = cosine_similarity(selected_elective_data, df_ratings.drop(columns=['electiva']))
    similar_indices = similarity_scores.argsort()[0][-top_n - 1:-1][::-1]
    similar_electives = df_ratings.iloc[similar_indices]['electiva'].tolist()

    return similar_electives

electiva_seleccionada = 'Innovación en servicios'
similares = find_similar_electives(electiva_seleccionada)
    
print(similares)

df_materias = df_materias.rename(columns={'class_name': 'electiva'})

df_completo = pd.merge(df_ratings, df_materias, on='electiva', how='inner')

print(df_completo.columns)

ddef obtener_recomendacion_usuario_nuevo(usuario_nuevo):
    recomendacion = {}

    for indice, fila in df_completo.iterrows():
        # Verificar si el usuario no ha visto la electiva
        if pd.isna(fila['general_rating']):
            # Calcular similitud entre el usuario y la electiva
            similitud = cosine_similarity([list(usuario_nuevo.values)], [list(fila[['general_rating', 'method_rating', 'personalGrowth_rating', 'rec_rating', 'teacher_rating', 'time_rating']].values)])[0][0]
            recomendacion[fila['electiva']] = similitud

    # Seleccionar la electiva con mayor similitud
    mejor_electiva = max(recomendacion, key=recomendacion.get)
    afinidad = recomendacion[mejor_electiva]

    return mejor_electiva, afinidad

# Usuario nuevo
nuevo_usuario = {'general_rating': 1, 'method_rating': 0, 'personalGrowth_rating': 1, 'rec_rating': 0, 'teacher_rating': 1, 'time_rating': 1}

# Obtener recomendación para el usuario nuevo
mejor_electiva_nuevo, afinidad_nuevo = obtener_recomendacion_usuario_nuevo(nuevo_usuario)

# Mostrar la recomendación para el usuario nuevo
print(f"Mejor recomendación para el usuario nuevo: {mejor_electiva_nuevo} (Afinidad: {afinidad_nuevo})")

SyntaxError: invalid syntax (1903063357.py, line 87)

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/rec-dmi', methods=['POST'])
def recibir_datos():
    datos_recibidos = request.json  
    print(datos_recibidos)
    respuesta = {"please":"help"}
    return jsonify(respuesta)

if __name__ == '__main__':
    app.run(port=5000) 